In [1]:
%cd /content/drive/MyDrive/Plivo/PII_NER_Project/pii_ner_assignment


/content/drive/MyDrive/Plivo/PII_NER_Project/pii_ner_assignment


In [2]:
!ls


assignment.md  generate_synthetic_data.py  README.md	     src
data	       out_baseline		   requirements.txt


In [2]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e5e2b52f4af769af9c3c80da193b514d2ad1b8ad88366547fb7639e72051403e
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [3]:
%%writefile generate_synthetic_data.py
import json, random, os

random.seed(42)

DIGIT_WORDS = ["zero","one","two","three","four","five","six","seven","eight","nine"]

FIRST_NAMES = ["rahul", "sneha", "arjun", "meera", "navin", "deepa", "akash", "ravi"]
CITIES = ["chennai", "mumbai", "delhi", "hyderabad", "bengaluru"]
LOCATIONS = ["iit madras campus", "city center", "airport road", "central station"]

DOMAINS = ["gmail dot com", "yahoo dot com", "outlook dot com"]

def make_phone():
    digits = [random.randint(0, 9) for _ in range(10)]
    return " ".join(DIGIT_WORDS[d] for d in digits)

def make_credit_card():
    digits = [random.randint(0,9) for _ in range(16)]
    groups = []
    for i in range(0, 16, 4):
        grp = " ".join(DIGIT_WORDS[d] for d in digits[i:i+4])
        groups.append(grp)
    return " ".join(groups)

def make_email():
    name = random.choice(FIRST_NAMES)
    return f"{name} at {random.choice(DOMAINS)}"

def make_date():
    days = ["first", "second", "third", "tenth", "twenty fifth"]
    months = ["january","february","march","april","may","june","july","august","september","october","november","december"]
    years = ["twenty twenty four","twenty twenty five","two thousand twenty four"]
    return f"{random.choice(days)} {random.choice(months)} {random.choice(years)}"

def make_example(i):
    label = random.choice(["CREDIT_CARD","PHONE","EMAIL","PERSON_NAME","DATE","CITY","LOCATION"])

    if label == "PHONE":
        value = make_phone()
        template = "my phone number is " + value
        start = len("my phone number is ")
        return template, [{"start": start, "end": start+len(value), "label":"PHONE"}]

    if label == "CREDIT_CARD":
        value = make_credit_card()
        template = "my credit card number is " + value
        start = len("my credit card number is ")
        return template, [{"start": start, "end": start+len(value), "label":"CREDIT_CARD"}]

    if label == "EMAIL":
        value = make_email()
        template = "my email is " + value
        start = len("my email is ")
        return template, [{"start": start, "end": start+len(value), "label":"EMAIL"}]

    if label == "PERSON_NAME":
        value = random.choice(FIRST_NAMES)
        template = "my name is " + value
        start = len("my name is ")
        return template, [{"start": start, "end": start+len(value), "label":"PERSON_NAME"}]

    if label == "DATE":
        value = make_date()
        template = "i will come on " + value
        start = len("i will come on ")
        return template, [{"start": start, "end": start+len(value), "label":"DATE"}]

    if label == "CITY":
        value = random.choice(CITIES)
        template = "ship this package to " + value + " city center"
        start = len("ship this package to ")
        return template, [{"start": start, "end": start+len(value), "label":"CITY"}]

    if label == "LOCATION":
        value = random.choice(LOCATIONS)
        template = "meet me at " + value
        start = len("meet me at ")
        return template, [{"start": start, "end": start+len(value), "label":"LOCATION"}]

def write_split(path, n, start_id=0, labeled=True):
    with open(path, "w") as f:
        for i in range(start_id, start_id+n):
            text, ents = make_example(i)
            if labeled:
                f.write(json.dumps({"id": f"utt_{i:04d}", "text": text, "entities": ents}) + "\n")
            else:
                f.write(json.dumps({"id": f"utt_{i:04d}", "text": text}) + "\n")

os.makedirs("data", exist_ok=True)

write_split("data/train.jsonl", 800, 0, True)
write_split("data/dev.jsonl", 150, 800, True)
write_split("data/test.jsonl", 100, 950, False)

print("Synthetic data generated successfully!")


Overwriting generate_synthetic_data.py


In [4]:
!python generate_synthetic_data.py


Synthetic data generated successfully!


In [5]:
!python src/train.py \
  --model_name distilbert-base-uncased \
  --train data/train.jsonl \
  --dev data/dev.jsonl \
  --out_dir out_baseline


tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 290kB/s]
config.json: 100% 483/483 [00:00<00:00, 3.28MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 5.68MB/s]
tokenizer.json: 100% 466k/466k [00:00<00:00, 2.90MB/s]
2025-11-23 13:52:23.384663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763905943.410269     898 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763905943.417770     898 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763905943.436895     898 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763905943.436942     898 computat

In [6]:
!python src/predict.py \
  --model_dir out_baseline \
  --input data/dev.jsonl \
  --output out_baseline/dev_pred.json


2025-11-23 14:02:08.954717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763906529.038144    3340 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763906529.063898    3340 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763906529.131841    3340 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906529.131931    3340 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906529.131942    3340 computation_placer.cc:177] computation placer alr

In [7]:
!python src/eval_span_f1.py \
  --gold data/dev.jsonl \
  --pred out_baseline/dev_pred.json


Per-entity metrics:
CITY            P=1.000 R=1.000 F1=1.000
CREDIT_CARD     P=1.000 R=1.000 F1=1.000
DATE            P=1.000 R=1.000 F1=1.000
EMAIL           P=1.000 R=1.000 F1=1.000
LOCATION        P=1.000 R=1.000 F1=1.000
PERSON_NAME     P=1.000 R=1.000 F1=1.000
PHONE           P=1.000 R=1.000 F1=1.000

Macro-F1: 1.000

PII-only metrics: P=1.000 R=1.000 F1=1.000
Non-PII metrics: P=1.000 R=1.000 F1=1.000


In [8]:
!python src/measure_latency.py \
  --model_dir out_baseline \
  --input data/dev.jsonl \
  --runs 50


2025-11-23 14:03:32.175807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763906612.204033    3697 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763906612.220340    3697 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763906612.256762    3697 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906612.256823    3697 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906612.256834    3697 computation_placer.cc:177] computation placer alr

In [9]:
!python src/train.py \
  --model_name prajjwal1/bert-tiny \
  --train data/train.jsonl \
  --dev data/dev.jsonl \
  --out_dir out_tiny


config.json: 100% 285/285 [00:00<00:00, 1.32MB/s]
vocab.txt: 232kB [00:00, 9.15MB/s]
2025-11-23 14:06:29.394389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763906789.435194    4419 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763906789.447109    4419 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763906789.475059    4419 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906789.475103    4419 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0

In [11]:
!python src/eval_span_f1.py \
  --gold data/dev.jsonl \
  --pred out_baseline/dev_pred.json


Per-entity metrics:
CITY            P=1.000 R=1.000 F1=1.000
CREDIT_CARD     P=1.000 R=1.000 F1=1.000
DATE            P=1.000 R=1.000 F1=1.000
EMAIL           P=1.000 R=1.000 F1=1.000
LOCATION        P=1.000 R=1.000 F1=1.000
PERSON_NAME     P=1.000 R=1.000 F1=1.000
PHONE           P=1.000 R=1.000 F1=1.000

Macro-F1: 1.000

PII-only metrics: P=1.000 R=1.000 F1=1.000
Non-PII metrics: P=1.000 R=1.000 F1=1.000


In [10]:
!python src/measure_latency.py \
  --model_dir out_tiny \
  --input data/dev.jsonl \
  --runs 50


2025-11-23 14:08:42.909458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763906922.934936    4988 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763906922.950464    4988 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763906922.979586    4988 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906922.979629    4988 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763906922.979635    4988 computation_placer.cc:177] computation placer alr

In [12]:
%%writefile generate_noisy_data.py
import json, random, os, string

random.seed(42)

DIGITS = ["zero","one","two","three","four","five","six","seven","eight","nine"]
MISHEAR = {
    "four": "for",
    "eight": "ate",
    "two": "to",
    "one": "won",
    "nine": "mine",
}
FILLERS = ["uh", "um", "err", "like"]
NOISE_WORDS = ["background", "noise", "static", "signal", "mic"]

FIRST_NAMES = ["rahul","arjun","meera","ravi","akhil","sneha","deepa"]
NAME_MISPELL = ["rahool","arjun","meerra","ravvi","akhill","snea","deepaa"]

CITIES = ["chennai","mumbai","delhi","hyderabad","bengaluru"]
CITY_MISSPELL = ["chenai","mumbay","delli","haidarabad","bangluru"]

DOMAINS = ["gmail dot com", "yahoo dot com", "outlook dot com"]

LOCATIONS = [
    "iit madras campus",
    "city center",
    "airport road",
    "central station"
]

def corrupt_word(word):
    # random spelling corruption
    if random.random() < 0.15:
        if len(word) > 3:
            i = random.randint(0, len(word)-2)
            return word[:i] + word[i+1:]  # delete one char
    # random merge
    if random.random() < 0.1:
        return word + random.choice(["a","e",""])
    return word

def noisy_digit():
    d = random.choice(DIGITS)
    if d in MISHEAR and random.random() < 0.2:
        d = MISHEAR[d]   # misrecognized digit
    return corrupt_word(d)

def make_phone():
    return " ".join(noisy_digit() for _ in range(10))

def make_credit_card():
    chunks = []
    for _ in range(4):
        chunk = " ".join(noisy_digit() for _ in range(4))
        if random.random() < 0.15:
            chunk += " " + random.choice(FILLERS)  # add filler noise
        chunks.append(chunk)
    return " ".join(chunks)

def make_email():
    name = random.choice(FIRST_NAMES)
    if random.random() < 0.25:
        name = random.choice(NAME_MISPELL)
    domain = random.choice(DOMAINS)
    # add random "dot" or "at" noise
    if random.random() < 0.2:
        domain = "gmaal dot com"
    return f"{name} at {domain}"

def make_date():
    days = ["first","second","third","tenth","twenty fifth"]
    months = ["january","february","march","april","may","june","july",
              "august","september","october","november","december"]
    years = ["twenty twenty four","twenty twenty five","two thousand twenty four"]
    st = f"{random.choice(days)} {random.choice(months)} {random.choice(years)}"
    # random filler noise
    if random.random() < 0.2:
        st = random.choice(FILLERS) + " " + st
    return st

def make_city():
    city = random.choice(CITIES)
    if random.random() < 0.3:
        city = random.choice(CITY_MISSPELL)
    return city

def make_location():
    loc = random.choice(LOCATIONS)
    if random.random() < 0.3:
        loc = corrupt_word(loc)
    return loc

def add_noise(text):
    words = text.split()
    # Random insert filler
    if random.random() < 0.3:
        idx = random.randint(0, len(words)-1)
        words.insert(idx, random.choice(FILLERS))
    # Random add noise words
    if random.random() < 0.2:
        words.append(random.choice(NOISE_WORDS))
    # Random merge or break words
    if random.random() < 0.2:
        i = random.randint(0, len(words)-2)
        words[i] = words[i] + words[i+1]
        del words[i+1]
    # Random missing space
    if random.random() < 0.2:
        i = random.randint(0, len(words)-2)
        words[i] = words[i] + words[i+1][0]
    return " ".join(words)

def make_example(i):
    label = random.choice(["PHONE","CREDIT_CARD","EMAIL","PERSON_NAME","DATE","CITY","LOCATION"])

    if label == "PHONE":
        value = make_phone()
        prefix = "my phone number is "
    elif label == "CREDIT_CARD":
        value = make_credit_card()
        prefix = "my credit card number is "
    elif label == "EMAIL":
        value = make_email()
        prefix = "my email is "
    elif label == "PERSON_NAME":
        value = random.choice(FIRST_NAMES)
        if random.random() < 0.3:
            value = random.choice(NAME_MISPELL)
        prefix = "my name is "
    elif label == "DATE":
        value = make_date()
        prefix = "i will come on "
    elif label == "CITY":
        value = make_city()
        prefix = "ship this package to "
    elif label == "LOCATION":
        value = make_location()
        prefix = "meet me at "

    text = prefix + value
    text = add_noise(text)

    start = text.find(value)
    end = start + len(value)

    return text, [{"start": start, "end": end, "label": label}]

def write_split(path, n, start=0, labeled=True):
    with open(path, "w") as f:
        for i in range(start, start+n):
            text, ents = make_example(i)
            ex = {"id": f"utt_{i:04d}", "text": text}
            if labeled:
                ex["entities"] = ents
            f.write(json.dumps(ex) + "\n")

os.makedirs("data_noisy", exist_ok=True)

write_split("data_noisy/train.jsonl", 800, 0, True)
write_split("data_noisy/dev.jsonl", 150, 800, True)
write_split("data_noisy/test.jsonl", 100, 950, False)

print("Generated REALISTIC noisy STT-style data!")


Writing generate_noisy_data.py


In [13]:
!python generate_noisy_data.py


Generated REALISTIC noisy STT-style data!


In [14]:
!python src/train.py \
  --model_name distilbert-base-uncased \
  --train data_noisy/train.jsonl \
  --dev data_noisy/dev.jsonl \
  --out_dir out_noisy


2025-11-23 14:20:43.949677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763907644.007129    7917 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763907644.024470    7917 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763907644.071607    7917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763907644.071699    7917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763907644.071705    7917 computation_placer.cc:177] computation placer alr

In [15]:
!python src/predict.py \
  --model_dir out_noisy \
  --input data_noisy/dev.jsonl \
  --output out_noisy/dev_pred.json


2025-11-23 14:34:16.066470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763908456.150346   11226 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763908456.174473   11226 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763908456.235973   11226 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763908456.236065   11226 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763908456.236076   11226 computation_placer.cc:177] computation placer alr

In [16]:
!python src/eval_span_f1.py \
  --gold data_noisy/dev.jsonl \
  --pred out_noisy/dev_pred.json


Per-entity metrics:
CITY            P=1.000 R=1.000 F1=1.000
CREDIT_CARD     P=0.100 R=0.148 F1=0.119
DATE            P=0.867 R=0.650 F1=0.743
EMAIL           P=0.571 R=0.286 F1=0.381
LOCATION        P=0.812 R=0.722 F1=0.765
PERSON_NAME     P=1.000 R=1.000 F1=1.000
PHONE           P=0.545 R=0.480 F1=0.511

Macro-F1: 0.646

PII-only metrics: P=0.519 R=0.509 F1=0.514
Non-PII metrics: P=0.925 R=0.881 F1=0.902


In [17]:
!python src/measure_latency.py \
  --model_dir out_noisy \
  --input data_noisy/dev.jsonl \
  --runs 50


2025-11-23 14:35:46.047934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763908546.089827   11605 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763908546.102675   11605 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763908546.132615   11605 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763908546.132696   11605 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763908546.132710   11605 computation_placer.cc:177] computation placer alr